In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('C://Users//seros//OneDrive//Desktop//T20 world cup project//info.csv')
df.head(n=5)

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


# while we are checking up with data do not have desired columns we are looking for

1. bowling_team
2. batting_team
3. city
4. current_score
5. balls_left
6. wickets_left
7. current_run_rate
8. last_five

## Now we will start our feature extraction from city column
## in order to start it we will first check for null values

In [4]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

# if we check above carefully the first word of venue column is actually representing the city Ex. Dubai In Dubai International Cricket Stadium

In [5]:
city = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [6]:
df['city'] = city

In [7]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

# we have filled city names in city column but we are aware that our data have approx 64000 columns we need to check how many balls are bowled and played

In [8]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,Melbourne Cricket Ground


In [9]:
df.drop('venue',axis='columns',inplace = True)
df.drop('Unnamed: 0',axis = 'columns',inplace = True)

In [10]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

# there are cities with very less deliveries we only consider those cities which have deliveries more than 600

In [11]:
df=df[df['city'].isin(eligible_cities)]

In [12]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne


In [13]:
df.shape

(50501, 7)

In [23]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

# Since we have extracted data from city column above we have used cumsum() function to extract current score 

In [24]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3


In [25]:
df['ball']


0         0.1
1         0.2
2         0.3
3         0.4
4         0.5
         ... 
63883    19.3
63884    19.4
63885    19.5
63886    19.6
63887    19.7
Name: ball, Length: 50501, dtype: float64

# After current score we need to extract balls_left from data before that we'd be creating columns over (how many overs are completed ) and balls(how many balls has been bowled). 

In [26]:
df['over'] = df['ball'].apply(lambda x: str(x).split(".")[0])
df['balls_no'] = df['ball'].apply(lambda x: str(x).split(".")[1])

In [27]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,balls_no
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5


# to calculate balls_bowled by simple formula balls_bowled = over * 6 + balls


In [28]:
df['balls_bowled']=(df['over'].astype('int')*6)+(df['balls_no'].astype('int'))

In [29]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,balls_no,balls_bowled
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5


# we are aware that there are 6*20=120 balls in a innings we are giving (wide,no balls) value of 0 by subtracting balls_bowled by 120 we will get balls_left

In [30]:
df['balls_left'] = 120-df['balls_bowled']

In [31]:
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)

In [32]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,balls_no,balls_bowled,balls_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,117,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,118,2
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,119,1
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,120,0


In [33]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0'else 1)

In [34]:
df['player_dismissed'] = df['player_dismissed'].astype('int')

In [36]:
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()

In [37]:
df['wickets_left'] = 10-df['player_dismissed']

# if player got out it will be 0 and we will replace names with 1 and apply cumsum() to calculate total wickets and than we will subtract it from 10 to get to wickets left

In [38]:
df['crr'] = df['current_score']/df['balls_bowled']

In [39]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,balls_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,0.333333
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,0.750000
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,0.600000


In [41]:
groups=df.groupby('match_id')

match_ids=df['match_id'].unique()
last_five=[]
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30)['runs'].sum().values.tolist())

# we will be having scores of last five overs

In [42]:
df['last_five'] = last_five

In [43]:
final_df=df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [44]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [45]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Australia,Sri Lanka,Melbourne,0,119,10,0.000000,NaN,168
1,Australia,Sri Lanka,Melbourne,0,118,10,0.000000,NaN,168
2,Australia,Sri Lanka,Melbourne,1,117,10,0.333333,NaN,168
3,Australia,Sri Lanka,Melbourne,3,116,10,0.750000,NaN,168
4,Australia,Sri Lanka,Melbourne,3,115,10,0.600000,NaN,168
...,...,...,...,...,...,...,...,...,...
50496,Sri Lanka,Australia,Colombo,125,3,2,1.068376,32.0,128
50497,Sri Lanka,Australia,Colombo,125,2,2,1.059322,32.0,128
50498,Sri Lanka,Australia,Colombo,125,1,1,1.050420,32.0,128
50499,Sri Lanka,Australia,Colombo,127,0,1,1.058333,33.0,128


# We have extracted all desired columns from data and named it final_df 

In [46]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        12024
runs_x               0
dtype: int64

In [47]:
final_df.dropna(inplace = True)

In [48]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

# we have dropped all null values from final_df

In [49]:
final_df = final_df.sample(final_df.shape[0])

# To avoid any kind of bias in final_df we have shuffled the data

In [50]:
final_df.head()

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
29427,Sri Lanka,Australia,Pallekele,47,82,8,1.236842,40.0,198
36592,Sri Lanka,Australia,Melbourne,135,8,6,1.205357,45.0,161
40227,South Africa,England,Chittagong,50,84,10,1.388889,44.0,196
26674,New Zealand,Australia,Christchurch,210,2,4,1.779661,65.0,214
9851,Australia,South Africa,Johannesburg,104,56,8,1.625000,31.0,196


In [51]:
from sklearn.model_selection import train_test_split
X = final_df.drop('runs_x',axis = 'columns')
Y = final_df['runs_x']

In [52]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

# Since we have completed data extraction now its time start model building for that i have splitted final_df into train and test data 

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [54]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [61]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(drop = 'first'),['batting_team','bowling_team','city'])
], 
 remainder='passthrough')

In [63]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler(with_mean=False)),
    ('step3',XGBRegressor(n_estimators=300,learning_rate=0.2,max_depth=12,random_state=1))

])

In [64]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first'),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', StandardScaler(with_mean=False)),
                ('step3',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              earl...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=12, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, random_state=1, ...))])

In [65]:
Y_pred = pipe.predict(X_test)

In [66]:
print(r2_score(y_test,Y_pred))
print(mean_absolute_error(y_test,Y_pred))

0.9861394087268817
1.7001399458569946


In [67]:
import pickle

In [68]:
pickle.dump(pipe,open("pipe.pkl","wb"))

In [69]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
29427,Sri Lanka,Australia,Pallekele,47,82,8,1.236842,40.0,198
36592,Sri Lanka,Australia,Melbourne,135,8,6,1.205357,45.0,161
40227,South Africa,England,Chittagong,50,84,10,1.388889,44.0,196
26674,New Zealand,Australia,Christchurch,210,2,4,1.779661,65.0,214
9851,Australia,South Africa,Johannesburg,104,56,8,1.625000,31.0,196
...,...,...,...,...,...,...,...,...,...
23270,New Zealand,Sri Lanka,Colombo,38,84,8,1.055556,33.0,141
48822,Australia,India,Chandigarh,154,1,4,1.294118,40.0,160
16481,South Africa,England,Cape Town,86,48,7,1.194444,37.0,154
11877,Australia,England,Southampton,48,75,7,1.066667,39.0,157
